<a href="https://colab.research.google.com/github/Shazizan/portfolio/blob/master/etl_vault_ps_csv_order_item.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Set Up Spark**

## **Install Java & PySpark**

Insight: As colab does not come with Java by default, and PySpark needs it.

In [6]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!pip install pyspark

## **Set JAVA_HOME environment variable**

Insight: This tells PySpark where Java is located.

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

## **Verify Installation above (Java & PySpark)**

In [8]:
!java -version
!python --version
!pip show pyspark

openjdk version "11.0.28" 2025-07-15
OpenJDK Runtime Environment (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)
Python 3.12.12
Name: pyspark
Version: 3.5.1
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: py4j
Required-by: dataproc-spark-connect


## **Start Spark Session**

Insight: PySpark’s read.csv() does not support reading from HTTP/HTTPS URLs directly. It only supports:

- Local file paths (/content/...)
- Distributed file systems (e.g. HDFS, S3, GCS)
- Mounted paths (e.g. DBFS in Databricks)

Therefore, when we want to pass a raw GitHub URL, Spark can’t handle it — it expects a file system protocol, not an HTTP stream.

Alternatively: We need to read data using pandas and then convert it into Spark.

In [12]:
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ETL_by_PySpark") \
    .getOrCreate()

spark

# **Extraction / Read data from Source Repo**

In [13]:
pdf = pd.read_csv("https://raw.githubusercontent.com/Shazizan/data/refs/heads/master/order_items.csv")
df = spark.createDataFrame(pdf)
df.show(5)

+------+--------+-------+----------+-----------------+----------+--------------------+--------------------+--------------------+--------------------+------------------+
|    id|order_id|user_id|product_id|inventory_item_id|    status|          created_at|          shipped_at|        delivered_at|         returned_at|        sale_price|
+------+--------+-------+----------+-----------------+----------+--------------------+--------------------+--------------------+--------------------+------------------+
| 97315|   67028|  53520|     14235|           263063| Cancelled|2023-01-03 11:14:...|                 NaN|                 NaN|                 NaN|0.0199999995529651|
| 44354|   30556|  24316|     14235|           119879|  Complete|2024-10-19 07:20:...|2024-10-21 01:13:...|2024-10-24 08:27:...|                 NaN|0.0199999995529651|
| 43903|   30246|  24063|     14235|           118670|Processing|2025-07-19 07:01:...|                 NaN|                 NaN|                 NaN|0.0199

# **Transformation**

In this case, I decided to do data cleaning
- check the null value / NaN
- Convert NaN into Null

In [14]:
df.count()

181248

Insight:
- null → Missing value (no data at all)
- NaN → Not-a-Number (only for numeric columns, e.g., float or double)

## **Check for nulls per column**

In [17]:
from pyspark.sql.functions import col, sum as _sum

df.select([
    _sum(col(c).isNull().cast("int")).alias(c)
    for c in df.columns
]).show()

+---+--------+-------+----------+-----------------+------+----------+----------+------------+-----------+----------+
| id|order_id|user_id|product_id|inventory_item_id|status|created_at|shipped_at|delivered_at|returned_at|sale_price|
+---+--------+-------+----------+-----------------+------+----------+----------+------------+-----------+----------+
|  0|       0|      0|         0|                0|     0|         0|         0|           0|          0|         0|
+---+--------+-------+----------+-----------------+------+----------+----------+------------+-----------+----------+



## **Count NaNs per column**

In [16]:
from pyspark.sql.functions import col, sum as _sum, isnan

df.select([
    _sum(isnan(col(c)).cast("int")).alias(c)
    for c in df.columns
]).show()

+---+--------+-------+----------+-----------------+------+----------+----------+------------+-----------+----------+
| id|order_id|user_id|product_id|inventory_item_id|status|created_at|shipped_at|delivered_at|returned_at|sale_price|
+---+--------+-------+----------+-----------------+------+----------+----------+------------+-----------+----------+
|  0|       0|      0|         0|                0|     0|         0|     63220|      117309|     162963|         0|
+---+--------+-------+----------+-----------------+------+----------+----------+------------+-----------+----------+



## **Observation**

Insight: Since this is an order tracking data:

- the columns - shipped_at, delivered_at, returned_at → some NaNs are expected (e.g., cancelled orders)
- We might not want to drop these rows completely. Instead, consider filling them with null and the Data Analyst can handle them during analysis.

## **Convert the NaN value into Null**

In [22]:
from pyspark.sql.functions import col, when

columns_to_check = ["shipped_at", "delivered_at", "returned_at"]

for c in columns_to_check:
    df = df.withColumn(c, when(col(c) == "NaN", None).otherwise(col(c)))

In [23]:
# Show top 5 rows
df.show(5)

# Check schema and nulls
df.printSchema()

# Count total rows
df.count()


+------+--------+-------+----------+-----------------+----------+--------------------+--------------------+--------------------+--------------------+------------------+
|    id|order_id|user_id|product_id|inventory_item_id|    status|          created_at|          shipped_at|        delivered_at|         returned_at|        sale_price|
+------+--------+-------+----------+-----------------+----------+--------------------+--------------------+--------------------+--------------------+------------------+
| 97315|   67028|  53520|     14235|           263063| Cancelled|2023-01-03 11:14:...|                NULL|                NULL|                NULL|0.0199999995529651|
| 44354|   30556|  24316|     14235|           119879|  Complete|2024-10-19 07:20:...|2024-10-21 01:13:...|2024-10-24 08:27:...|                NULL|0.0199999995529651|
| 43903|   30246|  24063|     14235|           118670|Processing|2025-07-19 07:01:...|                NULL|                NULL|                NULL|0.0199

181248

## **Save the transformed data into Parquet**

In [38]:
# Save as Parquet locally
df.write.parquet("cleaned_orders_item", mode="overwrite")

# **Load**

Load the parquet file into the gituhub repo (target system) using Github API

## **Install PyGitHub**

Insight: Only installs the library. Without it, we can’t use the GitHub API.

In [39]:
!pip install PyGithub

## **Import and Authenticate**

Insight: Lets Python “log in” to GitHub using our token. Without authentication, we cannot push files to the repo.

In [40]:
from github import Github, Auth

# Create authentication using Auth
auth = Auth.Token("YOUR_GITHUB_TOKEN")

# Pass auth object to Github
g = Github(auth=auth)

# Access your repo
repo = g.get_user().get_repo("pipeline-vault")

## **Upload or update a Parquet file**

Insight: This is the actual step that sends our Parquet file to GitHub.

In [41]:
import base64

# Local Parquet file in Colab
local_file_path = "cleaned_orders_item.parquet"

# Path in the GitHub repo
repo_file_path = "cleaned_orders_item.parquet"

# Read the file as bytes
with open(local_file_path, "rb") as f:
    content = f.read()

# Upload the file: create if not exists, otherwise update
try:
    # Check if file exists
    existing_file = repo.get_contents(repo_file_path)
    # Update existing file
    repo.update_file(
        path=existing_file.path,
        message="Update cleaned orders Parquet",
        content=content,
        sha=existing_file.sha
    )
    print("File updated successfully!")
except:
    # File does not exist → create it
    repo.create_file(
        path=repo_file_path,
        message="Add cleaned orders Parquet",
        content=content
    )
    print("File uploaded successfully!")

File uploaded successfully!


# **Verify the File into Target Repo**

the file should appear in the repo: pipeline-vault